<a href="https://colab.research.google.com/github/MinnDdu/deep_learning_practice/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transfer Learning - 전이 학습**

Visual Recognition 문제는 수많은 사람들이 이미 도전했던 문제임 

결과가 좋았던 Conv + Pooling 모델들이 이미 많이 존재 
-> 그 모델을 가져와서 개/고양이 분류문제 해결해보자

 

1~3. 구글 Inception V3모델과 weight파일 로드해서 합치기 

4. 학습금지 레이어 설정 

5. Inception 모델에서 중간에 원하는 레이어만 잘라오기 

6. 내 모델 만들고 (특히 출력층 레이어) Inception 모델과 연결

7. 학습시키기

In [ ]:
import os
# 기존 개, 고양이 구별 프로젝트를 구글에서 가져온 모델을 이용해보자
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q dogs-vs-cats-redux-kernels-edition.zip
!unzip -q train.zip -d .

dogs-vs-cats-redux-kernels-edition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
import os
import tensorflow as tf
import shutil
# 데이터셋 준비
os.mkdir('/content/dataset')
os.mkdir('/content/dataset/dog')
os.mkdir('/content/dataset/cat')

for i in os.listdir('/content/train/'):
    if 'dog' in i:
        shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)
    if 'cat' in i:
        shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(150, 150), #이미지 사이즈 조절
    batch_size=64,
    subset='training',
    validation_split=0.2,
    seed=1234
)

val_ds = train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(150, 150),
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)
# 데이터 전처리 시키기
# 전처리 함수
def preprocessing_func(i, answer):
    i = tf.cast(i/255.0, tf.float32)
    return i, answer

train_ds = train_ds.map(preprocessing_func)
val_ds = val_ds.map(preprocessing_func)



Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


▼ Google Colab에서 Inception 모델 weight이 담긴 h5파일 다운받기

In [ ]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

구글의 InceptionV3 model 불러오기

In [13]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# 다른 모델을 가져올땐 제작자가 정한 input_shape를 알아야함! or 첫 레이어의 input_shape 수정필요
# keras는 input_shape 수정가능하게 끔 파라미터로 설정가능
# include_top -> top layer 가져올거냐? 보통 top layer는 마지막 Dense layer를 말함
# weights -> w 가중치들 가져올거냐 말거냐 -> 위에서 따로 가중치값들 가져와서 여기선 None
inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)

# model + weight 파일
inception_model.load_weights('inception_v3.h5')

# inception_model.summary()
# 우리는 마지막레이어(top) 제외해서 가져왔기 때문에 마지막 Dense레이어른 없음
# 이제 여기다가 개, 고양이를 학습시키는데 inceptionV3 모델의 w값들을 업데이트 시키면 안됨!
# 따라서 학습금지레이어 설정

# inception_model.layers -> 레이어들
for i in inception_model.layers:
    # 레이어(i)의 학습가능성을 False로 만들기
    i.trainable = False

# 근데 가끔 기존모델을 아주 살짝만 학습 시키기도함 - Fine tuning
# Ex) inceptionV3 model에서 mixed6 레이어 다음부터는 학습시키기
# google 문서에선 unfreeze 라는 변수 사용함...
unfreeze = False 
for i in inception_model.layers:
    if i.name == 'mixed6':
        unfreeze = True
    if unfreeze:
        i.trainable = True

# 원하는 레이어만 뽑아서 내 레이어나 모델과 합치기도 가능
last_layer = inception_model.get_layer('mixed7') # -> 레이어 이름 mixed7인 레이어 가져옴
print(last_layer)
print(last_layer.output)
print(last_layer.output_shape)




KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
(None, 7, 7, 768)


In [17]:
import tensorflow as tf
# 구글 인셉션 모델과 내가 만든 레이어 합쳐보자

# 중간에 가져온 mixed7 레이어의 output shape(?, 7, 7, 768) -> 너무 복잡.. -> Flatten()

layer1 = tf.keras.layers.Flatten()(last_layer.output) # 가져온 레이어의 output
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

my_model = tf.keras.Model(inception_model.input, output_layer)

# optimizer=tf.keras.optimizers.Adam(lr=0.00001) 같이 learning_rate 아주 작게 설정시 w값들 미세하게 업데이트 가능
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
my_model.fit(train_ds, validation_data=val_ds, epochs=2) # 내가 만든 레이어들 학습시켜야함



Epoch 1/2
79/79 [==============================] - 362s 5s/step - loss: 0.3885 - acc: 0.9248 - val_loss: 0.0518 - val_acc: 0.9794
Epoch 2/2
79/79 [==============================] - 358s 5s/step - loss: 0.0528 - acc: 0.9808 - val_loss: 0.0085 - val_acc: 0.9978
